In [ ]:
import pandas as pd
import numpy as np
import string
import re
import nltk
nltk.download('punkt')
from google.colab import drive
from collections import defaultdict, Counter
from nltk.util import pad_sequence, ngrams, bigrams
from nltk import word_tokenize, sent_tokenize
import sys

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **1. N-gram**

## <font color=navy> **Preprocessing (1a)**
* Appears that sentences are divided by newline characters, so will split text into list of sentences; will use those to determine < s > and < /s > placement

* Text contains < unk > token; because one of N-gram model's issue is not being able to predict words not in vocabulary, I will keep this token

* Other preprocessing will just include normalizing (converting to all lower case)

In [ ]:
# Data processing function
def process_txt(file):
  with open('/content/drive/My Drive/' + file) as f:
    # Read as string
    text = f.read()

    # Normalize text by making all lowercase
    text = text.lower()

    # Split into sentences
    sentences = text.split('\n')

    # Add <s> and </s> tokens
    sentences = ['<s>' + s + '</s>' for s in sentences]

    # Return  sentences
    return sentences

In [ ]:
# Preprocess training and validation sets
train = process_txt('train.5k.txt')
validation = process_txt('valid.txt')

# Confirm 
print(train[0])
print(validation[0])

<s> aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter </s>
<s> consumers may want to move their telephones a little closer to the tv set </s>


Next, the sentences are tokenized. The tokenized lists are also used to create the training vocabulary.

In [ ]:
# Tokenize training sentences
train_tokenized = [sent.split(' ') for sent in train]

# Create vocabulary - join tokenized lists and then take set
all_tokens = []
for t in train_tokenized:
  all_tokens += t

vocabulary = list(set(all_tokens))

In [ ]:
# Get unigram counts
unigrams = dict(Counter(all_tokens))

In [ ]:
# Process input.txt
with open('/content/drive/My Drive/' + 'input.txt') as f:
    # Read as string
    input_text = f.read()

    # Get sequences
    input_sequences = input_text.split('\n')

    # Remove end line
    input_sequences = [' '.join(s.split(' ')[:-1]) for s in input_sequences]

In [ ]:
# Get first 30 lines of input sequences
input_sequences_30 = input_sequences[:30]

## <font color=navy> **N-gram model (1b)**

In [ ]:
# N-gram model
def train_ngram(sentences, n_val=2):
  # Define dictionary to hold probabilities, and initialize nested count values to 0
  counts = defaultdict(lambda: defaultdict(lambda: 0))
  ngram = defaultdict(lambda: defaultdict(lambda: 0))

  # Get count values (for Maximum Likelihood Estimate of conditional probabilities)
  if n_val == 3: # For trigram
    for s in sentences:
      print(s)
      for w0, w1, w2 in ngrams(s, n=3): # Unpack words
        # Update count (bigram prior > unigram)
        counts[(w0, w1)][w2] += 1 
        ngram[(w0, w1)][w2] += 1

  else: # For bigram or any other n value (error handling)
    for s in sentences:
      for w0, w1 in ngrams(s, n=2): # Unpack words
        # Update count (unigram prior > unigram)
        counts[w0][w1] += 1 
        ngram[w0][w1] += 1

  # Get probabilities 
  # Normalize numerator counts (either bigram or unigram - prior) by unigrams
  for prior in ngram:
    # Get unigram (denominator) total
    count_unigram = sum(ngram[prior].values())

    # Normalize each value (divide by unigram total)
    for u in ngram[prior]:
        ngram[prior][u] /= float(count_unigram)

  # Return mle estimation and counts (used for later smoothing)
  return pd.DataFrame.from_dict(ngram).fillna(0), pd.DataFrame.from_dict(counts).fillna(0)

In [ ]:
# Get ngram model and counts
mle, mle_counts = train_ngram(train_tokenized, n_val=2)

In [512]:
mle

,<s>,aer,banknote,berlitz,calloway,centrust,cluett,fromstein,gitano,guterman,hydro-quebec,ipo,kia,memotec,mlx,nahb,punts,rake,regatta,rubens,sim,snack-food,ssangyong,swapo,wachter,pierre,<unk>,<num>,years,old,will,join,the,board,as,a,nonexecutive,director,nov.,mr.,...,dealership,unrelated,laurel,intervene,adversary,inclined,reconsider,unfortunate,onerous,defend,brick,pre-trial,intact,innocent,comic,sand,await,charitable,whatever,newsprint,excellent,beatrice,scaled,reset,float,wastewater,uninsured,remic,20-year,weighted,j.c.,penney,railway,monte,di,deutsche,owe,minpeco,minerals,abramson
aer,0.0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pierre,0.0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mr.,0.0320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003912,0.001035,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.003676,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rudolph,0.0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a,0.0228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021515,0.024586,0.013158,0.022222,0.0,0.2,0.0,0.011765,0.147059,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a.g.,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s&ls,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
unexpectedly,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
proud,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000

## <font color=navy> **Good Turing smoothing (1c)**

In [ ]:
def train_gt_ngram(count):
  # Get count as an array
  gt_count = count.copy()
  count_array = gt_count.values

  # Define c* dictionary
  adjusted_count = count_array.copy()

  # Get counts of counts in count_array
  val, cnt = np.unique(count_array, return_counts=True)
  val = list(val)
  cnt = list(cnt)

  # Get N value (for Pgt denominator)
  N =sum(val[j]*cnt[j] for j in list(range(len(val))))

  # For every count c, compute adjusted count c*
  for i in [0,1]:#range(len(val) - 1):
    # Compute adjusted count c*
    c_star = (val[i] + 1)*cnt[i+1] / cnt[i]

    # Replace in adjusted_count
    adjusted_count = np.where(adjusted_count == val[i], c_star, adjusted_count)
  
  # Replace count values with adjusted counts
  gt_count[:] = adjusted_count

  # Get probability Pgt
  pgt = gt_count.copy()
  pgt.values /= N

  # Return Good Turing estimation and adjusted counts
  return pgt, gt_count    

In [ ]:
# Get GT estimation and counts
pgt, gt_counts = train_gt_ngram(mle_counts)

In [513]:
pgt

,<s>,aer,banknote,berlitz,calloway,centrust,cluett,fromstein,gitano,guterman,hydro-quebec,ipo,kia,memotec,mlx,nahb,punts,rake,regatta,rubens,sim,snack-food,ssangyong,swapo,wachter,pierre,<unk>,<num>,years,old,will,join,the,board,as,a,nonexecutive,director,nov.,mr.,...,dealership,unrelated,laurel,intervene,adversary,inclined,reconsider,unfortunate,onerous,defend,brick,pre-trial,intact,innocent,comic,sand,await,charitable,whatever,newsprint,excellent,beatrice,scaled,reset,float,wastewater,uninsured,remic,20-year,weighted,j.c.,penney,railway,monte,di,deutsche,owe,minpeco,minerals,abramson
aer,2.951658e-06,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,...,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09
pierre,2.951658e-06,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,...,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09
mr.,1.442429e-03,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,1.983340e-04,3.606073e-05,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,1.803036e-05,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,...,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09
rudolph,2.951658e-06,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190984e-09,7.190

## <font color=navy> **Kneser-Ney smoothing (1d)**

In [ ]:
# Specific to bigram
def train_kn_ngram(counts, unigram, d=0.75):
  # Make copy of counts for probability
  pkn = counts.copy()

  # Calculate number of unique bigrams
  num_unique_bigrams = sum((counts != 0).sum().values)

  # Iterate through every w_i-1 and wi pair
  i = 1 # For tracking training progress

  for prior in list(counts.columns):
    # Print progress
    sys.stdout.flush()
    sys.stdout.write('\rEvaluating prior (w_i-1) %d out of %d' % (i, pkn.shape[1]))

    for wi in list(counts.index):
      # Calculate bigram probability
      bigram_prob = max(counts.loc[wi, prior] - d, 0) / unigram[prior]

      # Calculate normalizing constant (discounted probability mass)
      lambda_prior = (d / unigram[prior]) * (counts.loc[:, prior] != 0).sum()

      # Calculate p(continuation) 
      # = number of words that wi follows / number of unique bigrams
      prob_continuation = (counts.loc[wi, :] != 0).sum() / num_unique_bigrams

      # Calculate pkn(wi|prior) and store in pkn DataFrame
      pkn.loc[wi, prior] = bigram_prob + (lambda_prior * prob_continuation)

    i += 1
  # Return Kneser-Ney probability estimation
  return pkn

<font color='maroon'>**Note:** Because of my nested for loop, this operation was expected to take 9 hours. I tried to thread it earlier, but Google Colab kept throwing RAM overload errors. It does run and the computation does seem correct, but unfortunately it was taking too long. In the future, I would eliminate the for loop and try to incorproate either a smaller vocabulary or vectorize the code more (i.e. use matrix operations).

In [ ]:
# Get PKN estimates
pkn = train_kn_ngram(mle_counts, unigrams)

Evaluating prior (w_i-1) 27 out of 7185

KeyboardInterrupt: ignored

## <font color=navy> **Predict next word in valid set (using sliding window) (1e)**

In [ ]:
# Prediction function (Bigram)
def ngram_predict(sequence, prob):
  # Get last word (w_i-1)
  prior = sequence.split(' ')[-1]

  # If prior is not vocabulary, get maximum probability based on '<unk>'
  if prior not in vocabulary:
    prior = '<unk>'

  # Get index of maximum probability and return (since it is a word)
  # Define wi probability values
  wi_probs = list(prob.loc[:, prior].values)

  # Since there could be ties, index returns first occurence
  return prob.index[wi_probs.index(max(wi_probs))]


In [ ]:
# For validation set, remove '</s>' token and also last word. The last word will be used for validation
test_clean = [v[4:-5] for v in validation]

# Create X sequence and y (last word in sequence)
X_test = [' '.join(x.split(' ')[:-1]) for x in test_clean]
y_test = [x.split(' ')[-1] for x in test_clean]

# Confirm processing
print(test_clean[0])
print(X_test[0])
print(y_test[0])

consumers may want to move their telephones a little closer to the tv set
consumers may want to move their telephones a little closer to the tv
set


In [ ]:
# Get predictions - MLE
mle_test_pred = [ngram_predict(X_test[i], mle) for i in [0, 1, 2, 3, 4]]

# Get predictions - Good Turing
good_turing_test_pred = [ngram_predict(X_test[i], pgt) for i in [0, 1, 2, 3, 4, 5]] # For some reason, list(range(len(X_test))) is failing

'''
# Get predictions - Kneser-Ney Smoothing
kn_test_pred = [ngram_preidct(X_test[i], pkn) for i in list(range(len(X_test)))]
'''

'\n# Get predictions - Kneser-Ney Smoothing\nkn_test_pred = [ngram_preidct(X_test[i], pkn) for i in list(range(len(X_test)))]\n'

In [ ]:
# Perplexity (Bigram)
def bigram_perplexity(prob, test_sequences, ground_truths):
  # Get list of priors
  priors = []

  # Add priors to list
  for sequence in test_sequences:
    # Get lislast word (w_i-1)
    prior = sequence.split(' ')[-1]

    # If prior is not vocabulary, get maximum probability based on '<unk>'
    if prior not in list(prob.columns):
      prior = '<unk>'

  priors.append(prior)

  # Take the product of all the inverse correct probabilities of (wi|wi_1)
  prod = 1

  for gt, p in list(zip(ground_truths, priors)):
    # Set ground truth value as '<unk>' if not in vocabulary
    if gt not in list(prob.index):
      gt = '<unk>'

    # Multipy (1/P(wi|w_i-1))
    prod *= (1 / prob.loc[gt, p])

  # Return the product to the 1/N power
  return prod**(1/len(priors))

In [ ]:
# Report perplexity scores on test set
print('MLE: ', bigram_perplexity(mle, X_test, y_test))
print('Good Turing: ', bigram_perplexity(pgt, X_test, y_test))

'''
print('Kneser-Ney: ', bigram_perplexity(pkn, X_test, y_test))
'''

MLE:  2812.0
Good Turing:  55462.0


"\nprint('Kneser-Ney: ', bigram_perplexity(pkn, X_test, y_test))\n"

Because the goal is to minimize the perplexity, it is not ideal that these perplexity scores are so high.

## <font color=navy> **Predict input_sequence next word (1f)**
Because I could not train the Kneser-Key N-gram model, I outputed the results for both the MLE and Good Turning estimates on the input sequence prediction.

In [ ]:
# Bigram MLE - Print the 30 sequences and their next words (specified in ** **)
print('MLE (BIGRAM) PREDICTIONS\n')
for i in range(30):
  # Get prediction
  pred = ngram_predict(input_sequences_30[i], mle)

  # Print completed sequence
  print(input_sequences_30[i] + ' ' + '**' + pred + '**')

MLE (BIGRAM) PREDICTIONS

but while the new york stock exchange did n't fall **in**
some circuit breakers installed after the october N crash failed **to**
the N stock specialist firms on the big board floor **traders**
big investment banks refused to step up to the plate **a**
heavy selling of standard & poor 's 500-stock index futures **prices**
seven big board stocks ual amr bankamerica walt disney capital **markets**
once again the specialists were not able to handle the **<unk>**
<unk> james <unk> chairman of specialists henderson brothers inc. it **is**
when the dollar is in a <unk> even central banks **</s>**
speculators are calling for a degree of liquidity that is **a**
many money managers and some traders had already left their **<unk>**
then in a <unk> plunge the dow jones industrials in **the**
<unk> trading accelerated to N million shares a record for **the**
at the end of the day N million shares were **n't**
the dow 's decline was second in point terms only **<num>**
in 

In [ ]:
# Good Turing - Print the 30 sequences and their next words (specified in ** **)
print('GOOD TURING PREDICTIONS\n')
for i in range(30):
  # Get prediction
  pred = ngram_predict(input_sequences_30[i], pgt)

  # Print completed sequence
  print(input_sequences_30[i] + ' ' + '**' + pred + '**')

GOOD TURING PREDICTIONS

but while the new york stock exchange did n't fall **in**
some circuit breakers installed after the october N crash failed **to**
the N stock specialist firms on the big board floor **traders**
big investment banks refused to step up to the plate **a**
heavy selling of standard & poor 's 500-stock index futures **prices**
seven big board stocks ual amr bankamerica walt disney capital **markets**
once again the specialists were not able to handle the **<unk>**
<unk> james <unk> chairman of specialists henderson brothers inc. it **is**
when the dollar is in a <unk> even central banks **</s>**
speculators are calling for a degree of liquidity that is **a**
many money managers and some traders had already left their **<unk>**
then in a <unk> plunge the dow jones industrials in **the**
<unk> trading accelerated to N million shares a record for **the**
at the end of the day N million shares were **n't**
the dow 's decline was second in point terms only **<num>**
in p

It appears that the MLE and Good Turing predictions are very similar, if not completely the same. For the most part, I think that the N-gram language modeling worked decently for these sequences, although there are for sure a few mistakes, and a significant amount of < unk > and < / s> predictions.

In [ ]:
'''
# Kneser-Ney N-gram - Print the 30 sequences and their next words (specified in ** **)
print('KNESER-NEY PREDICTIONS\n')
for i in range(30):
  # Get prediction
  pred = ngram_predict(input_sequences_30[i], pkn)

  # Print completed sequence
  print(input_sequences_30[i] + ' ' + '**' + pred + '**')
'''

"\n# Kneser-Ney N-gram - Print the 30 sequences and their next words (specified in ** **)\nprint('KNESER-NEY PREDICTIONS\n')\nfor i in range(30):\n  # Get prediction\n  pred = ngram_predict(input_sequences_30[i], pkn)\n\n  # Print completed sequence\n  print(input_sequences_30[i] + ' ' + '**' + pred + '**')\n"

# **2. RNN**

In [ ]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTMCell, Dense, GRU, Embedding
from keras.losses import SparseCategoricalCrossentropy
from keras.optimizers import Adam

## <font color=navy> **Parameter Initialization (2a)**

In [ ]:
# Parameters (used in 2d)
learning_rate = 1E-3
window_size = 20
batch_size = 50

## <font color=navy> **Further Preprocessing**

In [ ]:
# Remove <s> and </s> tokens from data and vocabulary (take second to last index because empty)
train_clean = [t[4:-5] for t in train][:-1]

# Update vocabulary
vocab_clean = vocabulary.copy()
vocab_clean.remove('<s>')
vocab_clean.remove('</s>')

# Encode the sequences using Keras integer mapping
train_encoded = [one_hot(s, len(vocab_clean)) for s in train_clean]

# Create X and y (last word in sequence)
X_train = [x[:-1] for x in train_encoded]
y_train = [x[-1] - 1 for x in train_encoded]

# Pad X_train
X_train = pad_sequences(X_train, maxlen=window_size)

In [ ]:
# Confirm
print(len(train_encoded))
print(X_train.shape)
print(len(y_train))

5000
(5000, 20)
5000


## <font color=navy> **Forward Pass (2b)**

In [ ]:
# Define model architecture
# Specify Sequential model
rnn = Sequential()

# Add Embedding (input) layer
# - input_dim = size of vocabulary
# - output_dim = batch_size
# - input length = window_size
rnn.add(Embedding(input_dim = len(vocab_clean), output_dim = batch_size, input_length=window_size))

# Add recurrent neural network cell (LSTM because good default choice)
rnn.add(LSTM(150))

# Add Dense layer 
rnn.add(Dense(len(vocab_clean), activation='softmax'))

# Print model summary
rnn.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_29 (Embedding)     (None, 20, 50)            359250    
_________________________________________________________________
lstm_27 (LSTM)               (None, 150)               120600    
_________________________________________________________________
dense_24 (Dense)             (None, 7185)              1084935   
Total params: 1,564,785
Trainable params: 1,564,785
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Define prediction function for next word
def predict_next_word(softmax_output, vocab):
  # Encode vocabulary
  vocab_encoded = [one_hot(v, len(vocab)) for v in vocab]

  # Get index of maximum input
  ind = np.argmax(softmax_output, axis=1)

  # Return encoded prediction vector and word prediction vector
  return ind, [vocab[i] for i in ind] 

In [ ]:
# Implement forward pass
forward_pass = rnn(X_train)

# Predict next word for all of the sequences
ind, word_pred = predict_next_word(forward_pass, vocab_clean)

In [ ]:
# Print predictions shape and first example of sequence + prediction to confirm
print(len(word_pred))
print()
print('EXAMPLE SEQUENCE PREDICTION:')
print('Predicted sequence: ' + X[1] + '[' + word_pred[1] + ']')
print('\nActual sequence: ' + X[1] + '[' + y[1] + ']')

5000

EXAMPLE SEQUENCE PREDICTION:
Predicted sequence: pierre <unk> n years old will join the board as a nonexecutive director nov.[peddling]

Actual sequence: pierre <unk> n years old will join the board as a nonexecutive director nov.[n]


## <font color=navy> **Calculate model loss (2c)**

In [ ]:
# Calculate sparse categorical cross entropy loss
initial_loss = SparseCategoricalCrossentropy()(y_train, forward_pass)
print(initial_loss)

tf.Tensor(8.879803, shape=(), dtype=float32)


## <font color=navy> **Setup training step (2d)**

In [ ]:
# Compile model
rnn.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy')

## <font color=navy> **Train RNN using training set (2e)**
* Trained for 100 epochs
* Used 10% of data for validation

In [ ]:
# Train/validation split
X_tr = X_train[:4500]
y_tr = np.asarray(y_train[:4500]).astype('float32')
X_val = X_train[4500:]
y_val = np.asarray(y_train[4500:]).astype('float32')

# Confirm shapes
print(X_tr.shape)
print(y_tr.shape)
print(X_val.shape)
print(y_val.shape)

(4500, 20)
(4500,)
(500, 20)
(500,)


In [ ]:
# Train RNN
history = rnn.fit(X_tr, y_tr, batch_size=batch_size, epochs=100, validation_data=(X_val, y_val))

Epoch 1/100
90/90 [==============================] - 7s 57ms/step - loss: 8.1891 - val_loss: 7.2873
Epoch 2/100
90/90 [==============================] - 5s 52ms/step - loss: 6.5534 - val_loss: 7.4023
Epoch 3/100
90/90 [==============================] - 5s 52ms/step - loss: 6.2948 - val_loss: 7.4303
Epoch 4/100
90/90 [==============================] - 5s 52ms/step - loss: 6.2041 - val_loss: 7.5456
Epoch 5/100
90/90 [==============================] - 5s 51ms/step - loss: 6.1897 - val_loss: 7.5703
Epoch 6/100
90/90 [==============================] - 5s 51ms/step - loss: 6.0669 - val_loss: 7.6191
Epoch 7/100
90/90 [==============================] - 5s 52ms/step - loss: 5.9447 - val_loss: 7.7048
Epoch 8/100
90/90 [==============================] - 5s 52ms/step - loss: 5.8907 - val_loss: 7.7752
Epoch 9/100
90/90 [==============================] - 5s 52ms/step - loss: 5.7725 - val_loss: 7.7974
Epoch 10/100
90/90 [==============================] - 5s 53ms/step - loss: 5.6970 - val_loss: 7.8623

Based on the validation loss, the model is overfitting within the first epoch. Nonetheless, I trained the model for 100 epochs to minimize the training less (although that is not at all good practice)!!

## <font color=navy> **Test RNN model / Calculate perplexity (2f)**

In [ ]:
# Preprocess validation data
# Remove <s> and </s> tokens from data
test_clean = [v[4:-5] for v in validation][:-1]

# Encode the sequences using Keras integer mapping
test_encoded = [one_hot(val, len(vocab_clean)) for val in test_clean]

# Create X and y (last word in sequence)
X_test = [x[:-1] for x in test_encoded]
y_test = [x[-1] - 1 for x in test_encoded]

# Pad X_test
X_test = pad_sequences(X_test, maxlen=window_size)

In [ ]:
# Confirm
print(len(test_encoded))
print(X_test.shape)
print(len(y_test))

3370
(3370, 20)
3370


In [ ]:
# Get predictions
test_softmax = rnn(X_test)

# Get test prediction index vector
test_ind_vec, test_word_vec = predict_next_word(test_softmax, vocab_clean)

In [ ]:
# Calculate model perplexity on test set
test_perplexity = 0

# Convert test_softmax to array
test_softmax = np.array(test_softmax)

for i in range(len(test_softmax)):
  # Get correct probability
  correct_prob = test_softmax[0][y_test[i]]

  # Add -log of correct_prob to test_perplexity
  test_perplexity += -np.log(correct_prob)

# Average and report
print('Test perplexity: ', np.exp(test_perplexity / len(test_softmax)))

Test perplexity:  47102243.3032223


That number seems very high!

## <font color=navy> **Perplexity proof (2g)**
* See other submission, 2g_Aprile_Allison.jpg

## <font color=navy> **Print input_sequence next word (2h)**

In [ ]:
# Preprocess input_sequence data

# Encode the sequences using Keras integer mapping
input_encoded = [one_hot(seq, len(vocab_clean)) for seq in input_sequences_30]

# Pad encoded sequences
X_input = pad_sequences(input_encoded, maxlen=window_size)

In [ ]:
# Confirm shapes
print(len(input_encoded))
print(X_input.shape)

30
(30, 20)


In [ ]:
# Get probability predictions for next word
input_preds = rnn(X_input)

# Get test prediction index vector
input_ind_vec, input_word_vec = predict_next_word(input_preds, vocab_clean)

In [ ]:
# Print the 30 sequences and their next words (specified in ** **)
for i in range(30):
  print(input_sequences_30[i] + ' ' + '**' + input_word_vec[i] + '**')

but while the new york stock exchange did n't fall **brushed**
some circuit breakers installed after the october N crash failed **eurodollars**
the N stock specialist firms on the big board floor **find**
big investment banks refused to step up to the plate **aspect**
heavy selling of standard & poor 's 500-stock index futures **damage**
seven big board stocks ual amr bankamerica walt disney capital **sleep**
once again the specialists were not able to handle the **publishers**
<unk> james <unk> chairman of specialists henderson brothers inc. it **'s**
when the dollar is in a <unk> even central banks **sentence**
speculators are calling for a degree of liquidity that is **mortgages**
many money managers and some traders had already left their **chemicals**
then in a <unk> plunge the dow jones industrials in **minneapolis**
<unk> trading accelerated to N million shares a record for **peddling**
at the end of the day N million shares were **market-share**
the dow 's decline was second in

Some of the above sequences are completed with reasonable words, although I feel that most of them are not logical. To improve this, I could do more text processing and add more layers/regularization to my model, as well as try out other embeddings with better bidirectionality (such as BERT), in the future.